In [121]:
import librosa
import soundfile
import os, glob, pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import itertools
from keras.backend import clear_session
from keras.models import Sequential
from keras import layers
from keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
import keras
import gensim.downloader
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import nltk

plt.style.use('ggplot')
#nltk.download('wordnet')
#nltk.download("stopwords")
stop_words = set(stopwords.words("english"))

def plot_history(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    x = range(1, len(acc) + 1)

    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(x, acc, 'b', label='Training acc')
    plt.plot(x, val_acc, 'r', label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.legend()
    plt.subplot(1, 2, 2)
    plt.plot(x, loss, 'b', label='Training loss')
    plt.plot(x, val_loss, 'r', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()

In [126]:
df=pd.read_csv("data.txt",sep=";")
print(set(df["emotion"]))
dic={'anger':0, 'fear':1, 'joy':2, 'love':3, 'sadness':4, 'surprise':5}
df['emotion']=[*map(dic.get, df['emotion'])]

{'fear', 'joy', 'surprise', 'love', 'sadness', 'anger'}


In [127]:
def remove_symbols(sentence):
	symbols="""!"#$%&'()*+,،-./:;<=>1234567890؟?@[\]^_`{|}~"""
	for sym in symbols:
		sentence.replace(sym,"")
	return sentence

def lowercase(sentence):
	return sentence.lower()

lemmatizer=WordNetLemmatizer()
def lemmatize(sentence):
	return " ".join([lemmatizer.lemmatize(word) for word in sentence.split(" ")])

def remove_stopwords(sentence):
	return " ".join([word for word in sentence.split(" ") if word not in stop_words])

df.sentence=df.sentence.apply(remove_symbols)
df.sentence=df.sentence.apply(lowercase)
df.sentence=df.sentence.apply(remove_stopwords)
df.sentence=df.sentence.apply(lemmatize)

def normalize_sentence(sent):
	return lemmatize(remove_stopwords(lowercase(remove_symbols(sent))))

In [143]:
index = df[df.duplicated() == True].index
df.drop(index, axis = 0, inplace = True)
df.reset_index(inplace=True, drop = True)
index = df[df.sentence.duplicated() == True].index
df.drop(index, axis = 0, inplace = True)
df.reset_index(inplace=True, drop = True)

In [145]:
print(df[df.sentence.duplicated() == True])
print(df[df.duplicated() == True])

Empty DataFrame
Columns: [sentence, emotion]
Index: []
Empty DataFrame
Columns: [sentence, emotion]
Index: []


In [146]:
#remove short sentences?

#inds=[]
#for ind in range(df.shape[0]):
	#if len(df.sentence[ind].split())<3:
	#	inds.append(ind)
#df.drop(inds, axis = 0, inplace = True)
#df.reset_index(inplace=True, drop = True)

In [147]:
sentences_train, sentences_test, emotion_train, emotion_test=train_test_split(df["sentence"],df["emotion"],test_size=.25)
y_train=emotion_train
y_test=emotion_test
y_train=to_categorical(y_train)
y_test=to_categorical(y_test)
tokenizer = Tokenizer(oov_token='UNK')
tokenizer.fit_on_texts(pd.concat([sentences_train, sentences_test], axis=0))
sequences_train=tokenizer.texts_to_sequences(sentences_train)
sequences_test=tokenizer.texts_to_sequences(sentences_test)
maxsentencelength=max([len(sent) for sent in df["sentence"]])
pad_sequences_train=pad_sequences(sequences_train,maxsentencelength)
pad_sequences_test=pad_sequences(sequences_test,maxsentencelength)
X_train=pad_sequences_train
X_test=pad_sequences_test

In [148]:
#glove_vectors = gensim.downloader.load('glove-wiki-gigaword-300')
dic_dim=len(tokenizer.word_index)
em_dim=len(glove_vectors["test"])

embedding_matrix=np.zeros((dic_dim+1,em_dim))

unk_count=0
unk_words=[]
unk_indices=[]
for word, i in tokenizer.word_index.items():
	try:
		embedding_matrix[i]=glove_vectors[word]
	except:
		unk_words.append(word)
		unk_count=unk_count+1
		unk_indices.append(i)
unk_indices=unk_indices[1:]
unk_words=unk_words[1:]

In [149]:
#Spellcheck!

#import textdistance
##spellchecked_indices=[np.argmin([textdistance.Jaccard(qval=2).distance(v,oov_word) for v in glove_vectors.index_to_key]) for oov_word in unk_words]
#spellchecked_indices=pd.read_csv("oov_indices.txt")
#spellchecked_indices=list(spellchecked_indices["0"])
#len(spellchecked_indices)
#spellchecked_words=[glove_vectors.index_to_key[ind] for ind in spellchecked_indices]
#for i in range(len(unk_indices)):
#	try:
#		embedding_matrix[unk_indices[i]]=glove_vectors[spellchecked_words[i]]
#	except:
#		pass

In [152]:
clear_session()
callback = EarlyStopping(
    monitor="val_loss",
    patience=4,
    restore_best_weights=True,
)
model = Sequential()
model.add(layers.Embedding(dic_dim+1, em_dim, input_length=X_train.shape[1], weights=[embedding_matrix], trainable=False))
model.add(layers.Bidirectional(layers.LSTM(20, recurrent_dropout=.2,return_sequences=True)))
model.add(layers.Bidirectional(layers.LSTM(10, recurrent_dropout=.2,return_sequences=True)))
model.add(layers.Bidirectional(layers.LSTM(10, recurrent_dropout=.2)))
model.add(layers.Dense(6, activation='sigmoid'))

model.compile(loss="CategoricalCrossentropy", 
            optimizer=keras.optimizers.Adam(learning_rate=.001), 
            metrics=['accuracy'])
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 229, 300)          4044000   
_________________________________________________________________
bidirectional (Bidirectional (None, 229, 40)           51360     
_________________________________________________________________
bidirectional_1 (Bidirection (None, 229, 20)           4080      
_________________________________________________________________
bidirectional_2 (Bidirection (None, 20)                2480      
_________________________________________________________________
dense (Dense)                (None, 6)                 126       
Total params: 4,102,046
Trainable params: 58,046
Non-trainable params: 4,044,000
_________________________________________________________________


In [153]:

history = model.fit(X_train, y_train,
                    epochs=200,
                    verbose=True,
                    validation_data=(X_test, y_test),
                    batch_size=50,
                    callbacks=[callback]
                    )
loss, accuracy = model.evaluate(X_train, y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_test, y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))
plot_history(history)

Epoch 1/200
239/239 [==============================] - 239s 932ms/step - loss: 1.5417 - accuracy: 0.4030 - val_loss: 1.0386 - val_accuracy: 0.6612
Epoch 2/200
239/239 [==============================] - 218s 914ms/step - loss: 0.9182 - accuracy: 0.6959 - val_loss: 0.6817 - val_accuracy: 0.7761
Epoch 3/200
239/239 [==============================] - 215s 902ms/step - loss: 0.5690 - accuracy: 0.8151 - val_loss: 0.5720 - val_accuracy: 0.8005
Epoch 4/200
239/239 [==============================] - 218s 912ms/step - loss: 0.4324 - accuracy: 0.8608 - val_loss: 0.4551 - val_accuracy: 0.8431
Epoch 5/200
239/239 [==============================] - 216s 905ms/step - loss: 0.3249 - accuracy: 0.8932 - val_loss: 0.4002 - val_accuracy: 0.8662
Epoch 6/200
239/239 [==============================] - 216s 906ms/step - loss: 0.2658 - accuracy: 0.9131 - val_loss: 0.3654 - val_accuracy: 0.8793
Epoch 7/200
239/239 [==============================] - 216s 902ms/step - loss: 0.2259 - accuracy: 0.9296 - val_loss: 0

In [ ]:
predicted = model.predict(X_test)
y_pred = predicted.argmax(axis=-1)
print(classification_report(emotion_test, y_pred))

              precision    recall  f1-score   support

           0       0.84      0.85      0.84        67
           1       0.88      0.81      0.84        57
           2       0.80      0.90      0.84       160
           3       0.67      0.55      0.60        44
           4       0.90      0.85      0.87       161
           5       0.60      0.55      0.57        11

    accuracy                           0.83       500
   macro avg       0.78      0.75      0.76       500
weighted avg       0.83      0.83      0.83       500

